In [1]:
from MNIST import MNIST_data, MNIST, MNIST_gridSearch

import torch 
import torch.nn as nn

import torch.nn.functional as F
import matplotlib.pylab as plt
import numpy as np

import os
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import torch.optim as optim

# from PIL import Image
import torchvision.transforms as transforms
import pandas as pd

import copy

torch.manual_seed(2)

# https://www.kaggle.com/competitions/digit-recognizer/overview

In [2]:
class PaulNet(nn.Module):
    def __init__(self):
        super(PaulNet, self).__init__()

        self.conv1 = nn.Conv2d(1, 8, kernel_size=5, stride = 1, padding = 0)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

        self.fc1 = nn.Linear(12*12*8, 56)
        self.fc2 = nn.Linear(56, 10)

    def forward(self, x):
        out = F.relu(self.conv1(x))
        out = self.pool1(out)
        out = out.view(-1, 12*12*8)

        out = F.relu(self.fc1(out))
        out = self.fc2(out)
        # out = F.softmax(out, dim=1)

        return out

In [3]:
batch_sizes = [8, 16, 32]
lrs = [0.0001, 0.0005]
epochs = [100, 400]
model = PaulNet()
criterions = [nn.CrossEntropyLoss]
optimizers = [optim.AdamW]

data = MNIST_data('train.csv', 'test.csv', 32, 0.8)
mnist = MNIST(data)
grid_search = MNIST_gridSearch(model, mnist, criterions, optimizers, epochs, lrs, batch_sizes)

best_param, best_model = grid_search.gridSearch(verbose = 2)
best_param

KeyboardInterrupt: 

In [ ]:
# [torch.optim.adamw.AdamW,
#  torch.nn.modules.loss.CrossEntropyLoss,
#  100,
#  0.0005,
#  16,
#  0.988452380952381]

### Submission

In [ ]:
# upcoming

### Running a test on my handwriting

In [ ]:
# upcoming (during this week, 18/11/24)